In [10]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
data = pd.read_excel('Cen_Data.xlsx')
import re
import jieba
jieba.load_userdict("./Cendict.txt")


def chinese_word_cut(mytext):
    #jieba.load_userdict('dic.txt')  # 这里你可以添加jieba库识别不了的网络新词，避免将一些新词拆开
    jieba.initialize()
    # 文本预处理 ：去除一些无用的字符只提取出中文出来
    new_data = re.findall('[\u4e00-\u9fa5]+', str(mytext), re.S)
    new_data = " ".join(new_data)

    # 文本分词
    seg_list_exact = jieba.cut(new_data, cut_all=True)
    result_list = []
    with open('Censtop'.txt','r', encoding='utf-8') as f:  # 可根据需要打开停用词库，然后加上不想显示的词语
        con = f.readlines()
        stop_words = set()
        for i in con:
            i = i.replace("\n", "")  # 去掉读取每一行数据的\n
            stop_words.add(i)

    for word in seg_list_exact:
        if word not in stop_words and len(word) > 1:
            result_list.append(word)
    return " ".join(result_list)


data["report_cutted"] = data.report.apply(chinese_word_cut)
#文本向量化
from sklearn.feature_extraction.text import CountVectorizer

n_features = 1000 # 提取1000个特征词语
tf_vectorizer = CountVectorizer(strip_accents='unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df=0.5,
                                min_df=10)
tf = tf_vectorizer.fit_transform(data.report_cutted)
#构建LDA
from sklearn.decomposition import LatentDirichletAllocation
n_topics = 13
lda = LatentDirichletAllocation(n_components=n_topics, max_iter=200,
                                learning_method='batch',
                                learning_offset=50,
                                doc_topic_prior=0.1,
                                topic_word_prior=0.01,
                               random_state=10000000)  # 关于模型的参数,可查看官方文档
lda.fit(tf)
#输出每个主题对应词语
def print_top_words(model, feature_names, n_top_words):
    tword = []
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic #{topic_idx}:" )
        topic_w = " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        tword.append(topic_w)
        print(topic_w)
    return tword
# 输出每个主题对应词语
n_top_words = 10
tf_feature_names = tf_vectorizer.get_feature_names_out()
topic_word = print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
绿化 生态环境 远海 自然 农药 合格 污染 兽药 监督 安全监管
Topic #1:
产业化 食品 流通 融资 供应 银行 贷款 消费 供应链 担保
Topic #2:
养护 厕所 绩效 专家 监督 整治 群众 村庄 试验 卫生
Topic #3:
补贴 补助 耕地 动物 农垦 机具 绩效 奖励 购置 粪污
Topic #4:
种植 大豆 玉米 产量 小麦 油菜 高产 种植业 农药 棉花
Topic #5:
脱贫 耕地 帮扶 农田 攻坚 保险 宅基地 三农 村庄 高标准
Topic #6:
机械 机械化 农业机械 初加工 机化 种植 机具 鉴定 技术装备 试验
Topic #7:
种植 设备 投资 集约 养殖场 蔬菜 水产 温室 保鲜 畜牧
Topic #8:
创业 就业 素质 农业社 高素质 技能 职业 农民工 返乡 人才队伍
Topic #9:
休闲 文化 旅游 创业 投资 产业园 园区 传统 价值 健康
Topic #10:
水产 种业 智慧 数字化 海洋 水产品 渔船 种质 政务 捕捞
Topic #11:
执法 行政 法治 用法 学法 法律 群众 基层 法规 纠纷
Topic #12:
农技 生物 农业科研 技术推广 院校 基层 绩效 防控 特聘 专家
